In [33]:
from lib_dataTransform import dataTransform

In [34]:
import pandas as pd

path=r'noExpired.xlsx'
x_all,y_all=dataTransform(path)

In [3]:
import pandas as pd

path=r'noExpired.xlsx'
all=pd.read_excel(path)
#numeric_cols = [ 'age', 'time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses']
numeric_cols = ['time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses']

x_num_all = all[ numeric_cols ].as_matrix()
x_num_all

array([[ 1, 35,  4, ...,  0,  0,  9],
       [ 2,  8,  5, ...,  0,  0,  6],
       [ 7, 12,  0, ...,  0,  1,  9],
       ..., 
       [13, 46,  2, ...,  0,  1,  6],
       [ 2, 62,  1, ...,  0,  3,  9],
       [ 8, 61,  0, ...,  0,  1,  9]], dtype=int64)

In [4]:
# categorical
cat_all = all.drop( numeric_cols + [ 'readmitted'], axis = 1 )

fac_x_cat_all = pd.DataFrame()
cat_cols = list(cat_all.columns.values)
for col in cat_cols:
    all_cur, _ = pd.factorize(cat_all[col])
    fac_x_cat_all[col] = all_cur

fac_x_cat_all = fac_x_cat_all.as_matrix()

import numpy as np
x_all = np.hstack(( x_num_all, fac_x_cat_all ))

In [5]:
y_all = all.readmitted

In [6]:
from sklearn.cross_validation import train_test_split
#train-test split
x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size=.25,random_state=24)

In [7]:

from sklearn import linear_model,datasets
#NOTE: change classifier here
logReg=linear_model.LogisticRegression(C=1e5)

In [8]:
import time
#training
st = time.time()
print "training started"
logReg.fit( x_train, y_train )
print "training ended"
et = time.time()
tt = et - st
print "Training Time = " + str(tt) + "\n"

logReg.score(x_train,y_train)

training started
training ended
Training Time = 0.476000070572



0.64708282333741329

In [9]:
#predictions
pred = logReg.predict( x_test )
pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [10]:
# generate class probabilities
probs=logReg.predict_proba(x_test)
probs

array([[ 0.62979799,  0.37020201],
       [ 0.62134139,  0.37865861],
       [ 0.7636931 ,  0.2363069 ],
       ..., 
       [ 0.59589486,  0.40410514],
       [ 0.69307152,  0.30692848],
       [ 0.61655918,  0.38344082]])

In [11]:
from sklearn import metrics
# generate evaluation metrics
print metrics.accuracy_score(y_test,pred)
print metrics.roc_auc_score(y_test,pred)
print metrics.confusion_matrix(y_test,pred)
print metrics.classification_report(y_test,pred)

0.648042414356
0.59727970169
[[1268  183]
 [ 680  321]]
             precision    recall  f1-score   support

          0       0.65      0.87      0.75      1451
          1       0.64      0.32      0.43      1001

avg / total       0.65      0.65      0.62      2452



In [12]:
from sklearn.cross_validation import cross_val_score
scores=cross_val_score(linear_model.LogisticRegression(),x_all,y_all,scoring='accuracy',cv=10)
print scores
print scores.mean()

[ 0.64525994  0.63608563  0.65239551  0.64627931  0.62996942  0.62959184
  0.65510204  0.66122449  0.64693878  0.65      ]
0.645284694918


In [13]:
# examine the coefficients
pd.DataFrame(zip(x_all.columns.values,np.transpose(logreg.coef_)))

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [23]:
from sklearn.metrics import precision_recall_curve, roc_curve, auc, confusion_matrix, average_precision_score

#PrecisionRecall-plot
precision = dict()
recall = dict()
PR_area = dict()
PR_thresholds = dict()
average_precision = dict()
for i in range(0):
    precision[i], recall[i], PR_thresholds[i] = precision_recall_curve(y_test[:,i],probs[:,1])
    PR_area[i] = auc(recall[i], precision[i])
    average_precision[i] = average_precision_score(y_test[:,i], probs[:, i])
# Compute micro-average ROC curve and ROC PR_area
precision["micro"], recall["micro"], _ = precision_recall_curve(y_test.ravel(), probs[:,1].ravel())
average_precision["micro"] = average_precision_score(y_test, pred, average='micro')


from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
pp = PdfPages('try3.pdf')
# Plot Precision-Recall curve for each class
plt.clf()
plt.plot(recall["micro"], precision["micro"], label='micro-average Precision-recall curve (PR_area = {0:0.2f})'.format(average_precision["micro"]))
for i in range(0):
    plt.plot(recall[i], precision[i], label='Precision-recall curve of class {' + mapping[i] + '}' + ' (PR_area = {1:0.2f})'.format(i, average_precision[i]))

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Extension of Precision-Recall curve to multi-class')
plt.legend(loc="lower right")
#plt.show()
plt.savefig(pp, format='pdf')

In [26]:
from sklearn.metrics import precision_recall_curve, roc_curve, auc, confusion_matrix, average_precision_score
#ROC-plot

def eer(fpr,tpr):
    dist=1000
    mi=0
    y1=1-fpr
    for j in range(0,(fpr.size-1)):
        di=abs(tpr[j]-y1[j])
        if(di < dist):
            dist=di
            mi=j
    return y1,mi


# Compute ROC curve and ROC ROC_area for each class
fpr = dict()
tpr = dict()
ROC_thresholds = dict()
ROC_area = dict()
for i in range(0):
    fpr[i], tpr[i], ROC_thresholds[i] = roc_curve(y_test[:, i], probs[:, 1])
    ROC_area[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC ROC_area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), probs[:, 1].ravel())
ROC_area["micro"] = auc(fpr["micro"], tpr["micro"])

fpr

{'micro': array([  0.00000000e+00,   0.00000000e+00,   6.89179876e-04, ...,
          9.98621640e-01,   9.99310820e-01,   1.00000000e+00])}

In [27]:
# Plot ROC curve
[y1,mi1]=eer(fpr[1],tpr[1])
plt.figure()
plt.plot(fpr["micro"], tpr["micro"], label='micro-average ROC curve (ROC_area = {0:0.2f})'.format(ROC_area["micro"]))
plt.plot(fpr[1],y1)
for i in range(0):
    plt.plot(fpr[i], tpr[i], label='ROC curve of class {' + mapping[i] + '}' + ' (ROC_area = {1:0.2f})'.format(i, ROC_area[i]))
    [y1,mi1]=eer(fpr[i],tpr[i])    
    plt.plot(fpr[i][mi1],y1[mi1],'o')
    print "EER => "+ mapping[i] + " =>" + str(mi1) + "#"

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")

#plt.show()
plt.savefig(pp, format='pdf')

KeyError: 1

In [28]:
i = np.arange(len(tpr)) # index for df
roc = pd.DataFrame({'fpr' : pd.Series(fpr, index=i),'tpr' : pd.Series(tpr, index = i), '1-fpr' : pd.Series(1-fpr, index = i), 'tf' : pd.Series(tpr - (1-fpr), index = i), 'thresholds' : pd.Series(thresholds, index = i)})
roc.ix[(roc.tf-0).abs().argsort()[:1]]

TypeError: unsupported operand type(s) for -: 'int' and 'dict'

In [31]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, pred)


0.59727970169045286